In [ ]:
#Import modules
import math
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import scipy.stats

In [ ]:
#Define the function b
def b(theta,x):
  y = 0
  if theta == 1:
    y = -1 + x - x**3
  if theta == 2:
    y = -1 + x - x**3
  return y

In [ ]:
#Define the function sigma
def sigma(theta,x):
  if theta == 1:
    y = 1 + (1+x)*(x**2)**(1/3)
  if theta == 2:
    y = x**4 + (x**4)**(1/3)
    y = 1 + math.sqrt(y/14)
  return y

In [ ]:
#Define the tamed function sigma
def smdelta(theta,x,delta):
  y = 1 + sigma(theta,x)/(1+delta**(1/2)*abs(sigma(theta,x)))
  return y

In [ ]:
#Define the step function h
def hdelta(x,delta):
  y = 10*delta/((1+abs(b(1,x))+abs(b(2,x))+abs(sigma(1,x))+abs(sigma(2,x))+(abs(x))**(2))**2)
  return y

In [ ]:
x0 = 0 #Initial value
T = 1 #Time length

In [ ]:
def kq(L, nMC):
    for l in range(2,L+1):
        saiso=0
        for iMC in range(1, nMC+1):
            #Define starting point the fine and coarse paths (Follow the idea in Fang & Giles 2020)
            t=0
            t_c=0
            t_f=0
            theta_old_c = 1
            theta_c = 1
            theta_old_f = 1
            theta_f = 1
            theta = 1
            h_c=0
            h_f=0
            dW_c=0
            dW_f=0
            X_c = x0
            X_f = x0
            n_c=2**l #Number of timestep in coarse path
            n_f=2**(l+1) #Number of timestep in fine path
            delta_c=1/n_c #Length of timestep in coarse path
            delta_f=1/n_f #Length of timestep in fine path
            while t<T:
                t_old = t
                theta_old_c = theta_c
                theta_old_f = theta_f
                t = min(t_c,t_f)
                dW = math.sqrt(t-t_old)*np.random.normal(0, 1)
                dW_c = dW_c+dW
                dW_f=dW_f+dW
                V = np.random.uniform(0,1)
                #Simulate the Markovian switching
                if theta == 1:
                  if V < 0.1*(t-t_old):
                    theta = 2
                else:
                  if V < 0.2*(t-t_old):
                    theta = 1
                if t==t_c:
                  X_c=X_c+b(theta_old_c,X_c)*h_c+smdelta(theta_old_c,X_c,delta_c)*dW_c   #update coarse path using h_c and W_c;
                  h_c=hdelta(X_c,delta_c)   #Compute new adapted coarse path timestep
                  h_c=min(h_c,T-t_c)   #Check the time point that is nearest to T
                  t_c=t_c+h_c        #update t_c
                  dW_c=0
                  theta_old_c = theta
                if t==t_f:
                  X_f=X_f+b(theta_old_f,X_f)*h_f+smdelta(theta_old_f,X_f,delta_f)*dW_f #update fine path using h_f and W_f;
                  h_f=hdelta(X_f,delta_f)   #Compute new adapted fine path timestep
                  h_f=min(h_f,T-t_f)    #Check the time point that is nearest to T
                  t_f=t_f+h_f     #update t_f
                  dW_f=0
                  theta_old_f = theta
            saiso=saiso+abs(X_c-X_f)
        Esaiso= math.log(saiso/nMC)/math.log(2)
        print(l, Esaiso)
    return None

In [ ]:
kq(6, 1000) #Show the value with l = 6 and the number of sample to compute the expectation of MSE is 1000